In [2]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Read the data in pandas dataframe

In [4]:
data = pd.read_csv('Project_Data_EE4C12_S&S_EMG.csv')

data = data.astype({
    'time': 'int64',
    'class': 'int64',
    'label': 'int64',
    'channel1': 'float64',
    'channel2': 'float64',
    'channel3': 'float64',
    'channel4': 'float64',
    'channel5': 'float64',
    'channel6': 'float64',
    'channel7': 'float64',
    'channel8': 'float64'
})

In [21]:
# Find the indices where the 'time' column resets to 0
reset_indices = data.index[data['time'] == 1].tolist()

# Split the data into separate DataFrames based on the reset indices
split_data = []

for i in range(len(reset_indices) - 1):
    split_data.append(data.iloc[reset_indices[i]:reset_indices[i + 1]])

# Append the last segment of data
split_data.append(data.iloc[reset_indices[-1]:])

# Display the number of segments
print(f"Number of segments: {len(split_data)}")

Number of segments: 58


In [6]:
# Count the number of segments for each label
label_counts = {}

for segment in split_data:
    if not segment.empty:
        label = segment['label'].iloc[0]
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

# Display the counts for each label
print(label_counts)

{1: 1, 2: 2, 3: 2, 4: 2, 5: 2, 6: 1, 7: 2, 8: 2, 9: 2, 10: 2, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 2, 17: 1, 18: 2, 19: 1, 20: 2, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 26: 2, 27: 1, 28: 1, 30: 1, 31: 2, 32: 2, 33: 2, 34: 2, 35: 2, 36: 2}


In [ ]:
# Plot the first segment in split_data
first_segment = split_data[0]

# Plotting the data
fig, axs = plt.subplots(8, 1, figsize=(12, 24), sharex=True)
channels = first_segment.columns[1:9]  # Exclude 'time' column

for i, column in enumerate(channels):
    axs[i].plot(first_segment['time'], first_segment[column])
    axs[i].set_title(column)
    axs[i].set_ylabel('Value')

plt.xlabel('Time')
plt.suptitle('First Segment Data')
plt.show()

In [23]:
print(type(split_data))
#split_data = pd.concat(split_data)
# Define the channels
channels = ['channel1', 'channel2', 'channel3', 'channel4', 'channel5', 'channel6', 'channel7', 'channel8']

# Function to calculate statistics
def calculate_statistics(df):
    stats = {
        'Mean': df.mean(),
        'Max': df.max(),
        'Min': df.min(),
        'Variance': df.var(),
        'MeanSquared': (df ** 2).mean(),
        'Power': (df ** 2).sum(),
        'Skewness': df.skew(),
        'Kurtosis': df.kurt(),
    }
    # Return as Series instead of DataFrame for scalar values
    return pd.Series(stats)

# Group by 'label' (assuming 'label' refers to each person/gesture)
statistics_by_label = {}


# Iterate through each segment and calculate statistics per channel for each
statistics_by_segment = {}

for i, segment in enumerate(split_data):
    stats_per_segment = {}
    for channel in channels:
        stats_per_segment[channel] = calculate_statistics(segment[channel])
    
    # Store the results in a dictionary with segment number as the key
    statistics_by_segment[f"Segment_{i+1}"] = pd.DataFrame(stats_per_segment)

# Concatenate all segment statistics into a single DataFrame
final_stats_df = pd.concat(statistics_by_segment, axis=0)

# Display the results for each segment
for segment in statistics_by_segment.keys():
    print(f"Statistics for {segment}:")
    print(statistics_by_segment[segment])
    print("-" * 50)








<class 'list'>
Statistics for Segment_1:
                 channel1      channel2      channel3      channel4  \
Mean        -7.865478e-06 -8.929025e-06 -9.766196e-06 -1.076941e-05   
Max          1.270000e-03  1.270000e-03  1.270000e-03  1.270000e-03   
Min         -1.160000e-03 -1.130000e-03 -1.060000e-03 -1.120000e-03   
Variance     1.829595e-08  1.745058e-08  1.434506e-08  2.294710e-08   
MeanSquared  1.835767e-08  1.753016e-08  1.444032e-08  2.306289e-08   
Power        2.224399e-03  2.124129e-03  1.749734e-03  2.794530e-03   
Skewness     4.581331e-02 -2.038683e-02  3.598835e-01  4.332024e-01   
Kurtosis     1.558864e+01  1.500314e+01  1.329044e+01  1.230872e+01   

                 channel5      channel6      channel7      channel8  
Mean        -1.398457e-05 -1.164463e-05 -9.673186e-06 -8.769745e-06  
Max          1.270000e-03  1.270000e-03  1.270000e-03  1.270000e-03  
Min         -1.280000e-03 -1.120000e-03 -1.090000e-03 -1.020000e-03  
Variance     3.727620e-08  2.190527e-08

In [26]:
from scipy.fft import fft
from scipy.signal import welch

import pandas as pd
import numpy as np
from scipy.fft import fft
from scipy.signal import welch

# Define the channels you're interested in
channels = ['channel1', 'channel2', 'channel3', 'channel4', 'channel5', 'channel6', 'channel7', 'channel8']

# Function to calculate Zero Crossing (ZC)
def calculate_zc(signal):
    zero_crossings = np.where(np.diff(np.signbit(signal)))[0]
    return len(zero_crossings)

# Function to calculate Slope Sign Changes (SSC)
def calculate_ssc(signal):
    ssc = np.sum(np.diff(np.sign(np.diff(signal))) != 0)
    return ssc

# Function to calculate Waveform Length (WL)
def calculate_wl(signal):
    wl = np.sum(np.abs(np.diff(signal)))
    return wl

# Other functions for calculating previously mentioned features (Fourier Transform, SDFT, PSD)
def calculate_ft(signal):
    ft = fft(signal)
    return np.abs(ft)

def calculate_sdft(signal, window_size):
    step_size = window_size // 2  # 50% overlap
    sdft_features = []
    for i in range(0, len(signal) - window_size, step_size):
        windowed_signal = signal[i:i+window_size]
        sdft = fft(windowed_signal)
        sdft_features.append(np.abs(sdft))
    return np.array(sdft_features)

def calculate_psd(signal, fs=1000):
    freqs, psd = welch(signal, fs)
    return freqs, psd

# RMS and Mean Absolute Value (MAV)
def calculate_rms(signal):
    return np.sqrt(np.mean(np.square(signal)))

def calculate_mav(signal):
    return np.mean(np.abs(signal))

# Set sampling frequency (for PSD and SDFT)
fs = 1000  # Example, adjust based on your dataset

# Extract features for each segment
features_by_segment = []

for segment in split_data:
    segment_features = {}
    for channel in channels:
        signal = segment[channel].values

        # Fourier Transform (FT)
        ft_features = calculate_ft(signal)
        segment_features[f'{channel}_ft_mean'] = np.mean(ft_features)
        segment_features[f'{channel}_ft_max'] = np.max(ft_features)

        # Short-Time Fourier Transform (SDFT)
        window_size = 256  # Example window size, adjust based on your needs
        sdft_features = calculate_sdft(signal, window_size)
        segment_features[f'{channel}_sdft_mean'] = np.mean(sdft_features)
        segment_features[f'{channel}_sdft_max'] = np.max(sdft_features)

        # Power Spectral Density (PSD)
        freqs, psd = calculate_psd(signal, fs)
        segment_features[f'{channel}_psd_mean'] = np.mean(psd)
        segment_features[f'{channel}_psd_max'] = np.max(psd)

        # Zero Crossing (ZC)
        zc = calculate_zc(signal)
        segment_features[f'{channel}_zc'] = zc

        # Slope Sign Changes (SSC)
        ssc = calculate_ssc(signal)
        segment_features[f'{channel}_ssc'] = ssc

        # Waveform Length (WL)
        wl = calculate_wl(signal)
        segment_features[f'{channel}_wl'] = wl

        # RMS (Root Mean Square)
        rms = calculate_rms(signal)
        segment_features[f'{channel}_rms'] = rms

        # MAV (Mean Absolute Value)
        mav = calculate_mav(signal)
        segment_features[f'{channel}_mav'] = mav

        # Additional features like Mean, Min, Max, etc.
        segment_features[f'{channel}_mean'] = np.mean(signal)
        segment_features[f'{channel}_min'] = np.min(signal)
        segment_features[f'{channel}_max'] = np.max(signal)
        segment_features[f'{channel}_variance'] = np.var(signal)

    # Append the extracted features for this segment
    features_by_segment.append(segment_features)

# Combine all extracted features into a DataFrame
features_df = pd.DataFrame(features_by_segment)

# Display the first few rows of the feature matrix
print(features_df.head())

# Optionally, save the feature matrix to a CSV file
features_df.to_csv('emg_extracted_features.csv', index=False)


   channel1_ft_mean  channel1_ft_max  channel1_sdft_mean  channel1_sdft_max  \
0          0.026558         0.953060            0.000809           0.060720   
1          0.026241         0.492300            0.000782           0.107030   
2          0.019576         0.506350            0.000720           0.059648   
3          0.031417         0.665380            0.001445           0.071200   
4          0.030588         0.726052            0.001602           0.089533   

   channel1_psd_mean  channel1_psd_max  channel1_zc  channel1_ssc  \
0       3.462692e-11      6.065651e-10         6351         17995   
1       5.592520e-11      8.348123e-10         4126         11503   
2       3.460340e-11      5.300794e-10         3659         10098   
3       1.032039e-10      1.776851e-09         3420          9330   
4       1.147347e-10      1.996065e-09         2951          7911   

   channel1_wl  channel1_rms  ...  channel8_psd_max  channel8_zc  \
0      1.26116      0.000135  ...      6.3

In [27]:
# Reorganize the DataFrame to have 'Segment', 'Channel', 'Feature', and 'Value' columns
reorganized_df = pd.melt(features_df, var_name='channel_feature', value_name='value')

# Split 'channel_feature' into 'Channel' and 'Feature'
reorganized_df['Channel'] = reorganized_df['channel_feature'].apply(lambda x: x.split('_')[0])
reorganized_df['Feature'] = reorganized_df['channel_feature'].apply(lambda x: '_'.join(x.split('_')[1:]))

# Drop the original 'channel_feature' column
reorganized_df = reorganized_df.drop(columns=['channel_feature'])

# Add a segment index (assuming each row is a separate segment)
reorganized_df['Segment'] = reorganized_df.index // len(channels)

# Reorder the columns to 'Segment', 'Channel', 'Feature', and 'Value'
reorganized_df = reorganized_df[['Segment', 'Channel', 'Feature', 'value']]

# Display the reorganized DataFrame
reorganized_df.head()




,channel1_ft_max,channel1_ft_mean,channel1_mav,channel1_max,channel1_mean,channel1_min,channel1_psd_max,channel1_psd_mean,channel1_rms,channel1_sdft_max,...,channel8_min,channel8_psd_max,channel8_psd_mean,channel8_rms,channel8_sdft_max,channel8_sdft_mean,channel8_ssc,channel8_variance,channel8_wl,channel8_zc
0,NaN,0.026558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.010656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.017419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0.020256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.033918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
